In [1]:
import os

In [2]:
%pwd

'/home/adam/Desktop/Projects/Kidney-Disease-Classification/research'

In [3]:
os.chdir("../")

Dagsub & MLFlow

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/amckenz/Kidney-Disease-Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="amckenz"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b349396f5f610b4d0b00a2fcba64d8ee7db5ae27"

In [5]:
import tensorflow as tf

2023-12-08 17:29:19.505742: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-08 17:29:19.507026: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-08 17:29:19.530398: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-08 17:29:19.531053: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 17:29:19.987597: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

Entity

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    # Everything from inside params.yaml
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    
    

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scans",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/amckenz/Kidney-Disease-Classification.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

create components

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    # from Keras site
    def valid_generator(self):
    
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)




    # ML Flow portion
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)       # Log all parameters
            mlflow.log_metrics(                             # Log all metrics
                {"loss":self.score[0], "accuracy": self.score[1]}
            )
            # Model Registry does not work with file store
            if tracking_url_type_store != "file":
                # Register model
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")

            else:
                mlflow.keras.log_model(self.model, "model")

Pipeline

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

    
except Exception as e:
    raise e

[2023-12-08 17:29:21,008: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-08 17:29:21,009: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-08 17:29:21,009: INFO: common: created directory at: artifacts]
Found 2207 images belonging to 2 classes.


2023-12-08 17:29:21.260918: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


138/138 [==============================] - 60s 433ms/step - loss: 18.1195 - accuracy: 0.6901
[2023-12-08 17:30:21,192: INFO: common: json file saved at: scores.json]


2023/12/08 17:30:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-12-08 17:30:22,516: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /tmp/tmpnv33h557/model/data/model/assets
[2023-12-08 17:30:22,847: INFO: builder_impl: Assets written to: /tmp/tmpnv33h557/model/data/model/assets]


/home/adam/Desktop/Projects/Kidney-Disease-Classification/.venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/12/08 17:30:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
